In [156]:
import numpy as np
import pandas as pd


data_folder  = '../data/titanic/preprocessed/'

In [157]:
X_train = pd.read_csv(data_folder + 'X_train.csv', index_col=False)
X_test = pd.read_csv(data_folder + 'X_test.csv', index_col=False)

In [168]:
import pickle

with open(data_folder + 'y_train.pkl', 'rb') as handle:
    y_train = pickle.load(handle)
    
with open(data_folder + 'y_test.pkl', 'rb') as handle:
    y_test = pickle.load(handle)

In [159]:
y_train.shape

(623,)

In [160]:
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.nn.functional as F

In [161]:
class LogisticRegressionModel(nn.Module):
    def __init__(self, input_dim, output_dim):
        super().__init__()
        self.linear = nn.Linear(input_dim, output_dim)
        
    def forward(self, x):
        x = self.linear(x)
        output = F.sigmoid(x)
        
        return output

In [162]:
input_dim = 10
output_dim = 1

model = LogisticRegressionModel(input_dim, output_dim)

In [163]:
print(len(list(model.parameters())))
print(list(model.parameters())[0].size())
print(list(model.parameters())[1].size())

2
torch.Size([1, 10])
torch.Size([1])


In [164]:
criterion = nn.BCELoss()
learning_rate = 0.001
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [169]:
y_train = y_train.values.reshape(-1, 1)

In [170]:
X_train.shape, y_train.shape

((623, 10), (623, 1))

In [171]:
num_epochs = 100000

In [172]:
for epoch in range(num_epochs):
    x_torch = torch.from_numpy(X_train.values).type(torch.FloatTensor)
    y_torch = torch.from_numpy(y_train).type(torch.FloatTensor)
    x = Variable(x_torch)
    y = Variable(y_torch)

    optimizer.zero_grad()

    outputs = model(x)
    cost = criterion(outputs, y)
    cost.backward()

    optimizer.step()    

In [173]:
y_pred = model(Variable(torch.from_numpy(X_test.values).type(torch.FloatTensor)))

In [174]:
y_pred = y_pred.data.numpy()

In [175]:
from sklearn.metrics import roc_auc_score

print(roc_auc_score(y_test, y_pred.reshape(-1, )))

0.8340277777777778


In [178]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression()

lr.fit(X_train, y_train.reshape(-1, ))
y_pred = lr.predict_proba(X_test)

In [179]:
print(roc_auc_score(y_test, y_pred[:,1]))

0.8317708333333333
